### 6.1 Maximum amount of gold

- Given a set of gold bars of various weights and a backpack that can hold at most W pounds, place as much gold as possible into the backpack.
    - **Input:** A set of $n$ gold bars of integer weights $w_1, ... w_n$ and a backpack that can hold at most $W$ pounds
    - **Output:** A subset of gold bars of maximum total weight not exceeding $W$
    - **Constraints:** $1 \le W \le 10^4$; $1 \le n \le 300$; $0 \le w_1, ... w_n \le 10^5$
    - **Sample:**
        - W = 10, (1,4,8) --> 9

In [ ]:
## inputs 
capacity = 10
weights = [1,4,8]

def maximum_gold(capacity, weights):
    ## build cache
    cache = [[0 for _ in range(capacity+1)] for _ in range(len(weights)+1)]

    ## build optimal weights row-wise.
    ## Each row shows the optimal weight of the items up to that point (i.e. in row 2, i only consider the first 2 elements)
    for row in range(1, len(weights)+1):
        for col in range(1, capacity+1):
            # print('='*50)
            # print(row, col)
            # display(cache)
            if col < weights[row-1]:
                cache[row][col] = cache[row-1][col]
            else:
                cache[row][col] = max(
                    cache[row-1][col-weights[row-1]] + weights[row-1],
                    cache[row-1][col]
                )

    ## return bottom right
    return cache[len(weights)][capacity]

maximum_gold(capacity, weights)

### 6.2 Splitting the Pirate Loot

- Partition a set of integers into three subsets with equal sums.
    - **Input:** A sequence of integers $v_1, ... v_n$
    - **Output:** Check whether it is possible to partition them into three subsets with equal sums, i.e. check whether there exist three disjoint sets $S_1, S_2, S_3 \subseteq \{1,2,...n\}$ such that $S_1 \cup S_2 \cup S_3 = \{1,2,...n\}$  and $$\sum_{i \in S_1} v_i = \sum_{j \in S_2} v_j = \sum_{k \in S_3} v_k$$
    - **Constraints:** $1 \le n \le 20$; $1 \le v_i \le 30$ for all $i$
    - **Samples:**
        - 4, [3,3,3,3] -> 0
        - 1, [30] -> 0
        - 13, [1,2,3,4,5,5,7,7,8,10,12,19,25] -> 1

- This is an interesting problem, because it can be solved via backtracking or bitmask

- Since this section is about dynamic programming, let's first try to solve via backtracking

In [113]:
values = [3,1,1,4,4,2]
partition_count=3

def partition3_iter(values, partition_count=3, verbose=False):
    total_sum = sum(values)
    if total_sum % partition_count != 0:
        return 0
        
    partition_value_target = total_sum/partition_count
    # values = sorted(values, reverse=True)
    partition_values = [0] * partition_count

    ## Each queue element represents (`value_index`, `partition_index`)
    queue = [(0, 0)]
    success = []
    total_succeed = 0
    i = 0
    while queue:
        i+=1
        if verbose:
            print('='*50)
            print(f"{queue=}, {success=}, {partition_values=}, {total_succeed=}, {i=}")

        value_index, partition_index = queue.pop()
        if value_index >= len(values):
            return 1

        if ((partition_values[partition_index] + values[value_index]) <= partition_value_target):
            if verbose:
                print(f'Successfully adding {value_index=}, {partition_index=}')
            partition_values[partition_index] += values[value_index]
            queue.append((value_index+1, 0))
            success.append((value_index, partition_index))
            if partition_values[partition_index] == partition_value_target:
                total_succeed += 1
        else:
            if verbose:
                print(f'Unable to add {value_index=}, {partition_index=}')
            at_partition_end = (partition_index+1) >= partition_count
            
            # Otherwise, try the current value in the next partition
            if not at_partition_end:
                if verbose:
                    print(f'Trying ({value_index=}, {partition_index+1=})')
                queue.append((value_index, partition_index+1))
            else:
                # If we are at the end of either the values or the partitions arrays, backtrack to last success
                if verbose:
                    print('At partition end')
                while at_partition_end:
                    try:
                        value_index, partition_index = success.pop()
                    except:
                        return 0
                    
                    if verbose:
                        print(f'Backtracking to ({value_index=}, {partition_index=})')
                    
                    if partition_values[partition_index] == partition_value_target:
                        total_succeed -= 1
                    
                    partition_values[partition_index] -= values[value_index]
                    
                    at_partition_end = (partition_index+1) >= partition_count
                    
                    ## If we have backtracked to the point where a partition value is 0, that means that no combination of values (from and including this value to the end) can give the desired value. 
                    ## Take for example the [4,4,2] subarray in [3,1,1,4,4,2]. 
                    ##  State: [5, 0, 0] 
                    ##  Target value: 5
                    ##  Subarray: [4,4,2]
                    ## We know that this is bound to fail. But before it fails, we must have tried 4+4, 4+2, 4+2 (that is, a+b, a+c, b+c)
                    ## Then, once all combinations are tried, there is no need to worry about the cases (b+a, c+a, c+b) by symmetry. So if we backtrack till a 0 is hit, break the iteration
                    if partition_values[partition_index] == 0:
                        at_partition_end = True

                queue.append((value_index, partition_index+1))
        
        if total_succeed >= (partition_count-1):
            return 1

partition3(values, verbose=False)

1

In [126]:
values = [3,1,1,4,4,2]
partition_count=3

def partition3_recurs(values, partition_count=3, verbose=False):
    total = sum(values)
    if total % partition_count != 0:
        return 0

    partition_value_target = total/partition_count
    partition_values = [0] * partition_count

    def try_place_value_at_index(i):
        # Index does not exist, return 
        if i >= len(values):
            return 1

        for partition_index in range(partition_count):
            if partition_values[partition_index] + values[i] <= partition_value_target:
                partition_values[partition_index] += values[i]

                if try_place_value_at_index(i+1):
                    return 1
                else:
                    partition_values[partition_index] -= values[i]
                    if partition_values[partition_index] == 0:
                        break
                              
        return 0
    return try_place_value_at_index(0)

partition3_recurs(values=values)

1

### 6.3 Maximum Value of an Arithmetic Expression

- Parenthesize an arithmetic expression to maximize its value
    - **Input:** An arithmetic expression consisting of digits as well as plus, minus, and multiplication signs
    - **Output:** Add parentheses to the expression in order to maximize its value
    - **Constraints:** $0 \le n \le 14$ (hence the string contains at most 29 symbols)
    - **Sample:** 
        - `5-8+7*4-8+9` -> `200`

In [ ]:
import re
import math

dataset = '5-8+7*4-8+9'

def maximum_value(dataset):
    digits = re.split('\-|\+|\*', dataset)
    ops = re.findall('\-|\+|\*', dataset)
    
    # every row represents a `start_index`, and every col represents an `end_index`
    # The value at (`start_index`, `end_index`) refers to the maximum possible value looking at the subarray of digits between the start and end indices
    # Since we have operations `+`, `-`, `*`, we need to store both the minimum and maximum possible values. Because, for example, a `-` operation is only maximised when the LHS subarray is maximised, and the RHS is minimised (assuming it is not negative)
    min_cache = [['' for _ in range(len(digits))] for _ in range(len(digits))]
    max_cache = [['' for _ in range(len(digits))] for _ in range(len(digits))]
    
    # We iterate through the cache diagonally, because at each value, we either fill it with the number that is directly above it (do not use the i-th value) or the one that is 1 row up and value(i) columns to the left
    for i in range(len(digits)):
        for j in range(len(digits)-i):
            # print('='*50)
            # print(f"{i=}, {j=}, {i+j=}")
            
            ## If we look at subarray of length 1, the only possible value is itself. 
            if j == i+j:
                min_cache[j][i+j] = digits[j]
                max_cache[j][i+j] = digits[j]
            
            ## Otherwise, if there is more than 1 digit, there is at least 1 way to add brackets. The idea here is that, for any composite array with 3 digits and above, you can split it into smaller subarrays with known values (e.g. with 3, you can split 1-2 or 2-1. With 4, you can split 1-3, 2-2, 3-1, and the 3 can be further split iteratively). The point is that the subarrays are of known values, so to find how to maximise any larger subarray, we just need to try 4 possible permutations of min and max for both the left and right subarrays
            else:
                minval = math.inf
                maxval = -math.inf

                for midpoint in range(j, i+j):
                    # print(f"{i=}, {j=}, {i+j=}, {midpoint=}")

                    minmin = eval(str(min_cache[j][midpoint]) + ops[midpoint] + str(min_cache[midpoint+1][i+j]))
                    maxmax = eval(str(max_cache[j][midpoint]) + ops[midpoint] + str(max_cache[midpoint+1][i+j]))
                    minmax = eval(str(min_cache[j][midpoint]) + ops[midpoint] + str(max_cache[midpoint+1][i+j]))
                    maxmin = eval(str(max_cache[j][midpoint]) + ops[midpoint] + str(min_cache[midpoint+1][i+j]))

                    minval = min(
                        minval,
                        minmin,
                        maxmax,
                        minmax,
                        maxmin, 
                    )

                    maxval = max(
                        maxval,
                        minmin,
                        maxmax,
                        minmax,
                        maxmin, 
                    )

                min_cache[j][i+j] = minval
                max_cache[j][i+j] = maxval

    return max_cache[0][len(digits)-1]

maximum_value(dataset)